In [1]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as pdr

from tqdm import tqdm_notebook , tqdm
from collections import deque

In [2]:
tf.__version__

'2.4.0'

# Construct AI

In [3]:
class AI_Trader():
  def __init__(self, state_size, action_space=3, model_name="AITrader"):
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.model_name= model_name
    self.gamma = 0.95
    self.epsilon = 1
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    self.model = self.modelBuilder()

  def modelBuilder(self):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=32,activation='relu',input_dim= self.state_size))
    model.add(tf.keras.layers.Dense(units=64,activation='relu'))
    model.add(tf.keras.layers.Dense(units=128,activation='relu'))
    model.add(tf.keras.layers.Dense(units=self.action_space,activation='linear'))
    model.compile(loss='mse',optimizer = tf.keras.optimizers.Adam(learning_rate=0.001))
    return model
  
  def trade(self,state):
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)

    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  def batch_train(self,bath_size):
    batch = []
    for i in range(len(self.memory) - batch_size+1,len(self.memory)):
      batch.append(self.memory[i])

    for state,action,reward,nextState,done in batch:
      if not done:
        reward = reward + self.gamma * np.argmax(self.model.predict(next_state)[0])

      target = self.model.predict(state)
      target[0][action] = reward

      self.model.fit(state, target,epochs=1, verbose=False)
    
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay



## Pre-Processing DataBa
se

## Sigmoid Function

In [4]:
def sigmoid(x):
  return 1/(1 + math.exp(-x))

In [5]:
sigmoid(0.5)

0.6224593312018546

## Price Format

In [6]:
def stocks_price_format(n):
  return "- ${0:2f}".format(abs(n)) if n < 0 else "${0:2f}".format(abs(n))

In [7]:
stocks_price_format(100)

'$100.000000'

## Load DataBase

In [8]:
dataset = pdr.DataReader('AAPL',data_source='yahoo')

In [9]:
dataset.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-01-15,24.427500,23.840000,24.049999,24.282499,319335600.0,22.497040
2016-01-19,24.662500,23.875000,24.602501,24.165001,212350800.0,22.388184
2016-01-20,24.547501,23.355000,23.775000,24.197500,289337600.0,22.418295
2016-01-21,24.469999,23.735001,24.264999,24.075001,208646000.0,22.304800
2016-01-22,25.365000,24.592501,24.657499,25.355000,263202000.0,23.490686


In [10]:
str(dataset.index[0]).split()[0]

'2016-01-15'

In [11]:
str(dataset.index[-1]).split()[0]

'2021-01-12'

In [12]:
def datasetLoader(stock_name):
  dataset = pdr.DataReader(stock_name,data_source='yahoo')
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  close = dataset['Close']
  return close

## Ambient States

In [13]:
def stateCreator(data,timestep,window_size):
  starting_id = timestep - window_size + 1
  window_data = data[starting_id:timestep+1] if (starting_id >= 0) else  - starting_id * [data[0]] + list(data[0:timestep+1])

  state = []

  for i in range(window_size-1):
    state.append(sigmoid(window_data[i+1]-window_data[i]))
  
  return np.array([state])

## Load Database

In [14]:
stock_name = 'AAPL'
data = datasetLoader(stock_name=stock_name)

In [15]:
s = stateCreator(data=data,timestep=20,window_size=5)

In [16]:
s

array([[0.45512103, 0.46443518, 0.51811712, 0.65982184]])

# Train

## Hyper-Params

In [17]:
window_size = 10
episodes = 1000
batch_size = 32
data_samples = len(data-1)

In [18]:
trader = AI_Trader(window_size)

In [19]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for episode in range(1,episodes+1): 
  print(f"Episode: {episode}/{episodes}")
  state = stateCreator(data,0,window_size + 1)
  total_profit = 0
  trader.inventory = []
  for t in tqdm(range(data_samples)):
    action = trader.trade(state=state)
    next_state = stateCreator(data,t+1,window_size+1)
    reward = 0

    if action == 1:
      #Buy Stock
      trader.inventory.append(data[t])
      print(f'AI Trader bought: {stocks_price_format(data[t])}')
    elif action == 2 and len(trader.inventory) > 0:
      buyPrice = trader.inventory.pop(0)
      reward = max(data[t]-buyPrice,0)
      totalProfit = data[t] - buyPrice
      print(f'AI Trader sold: {data[t]} Profit: {stocks_price_format(data[t]-buyPrice)}')
    if t == data_samples - 1:
      done = True
    else: 
      done = False
    
    trader.memory.append((state,action,reward,next_state,done))
    state = next_state

    if done:
      print('#############')
      print('#############')
      print(f'Total Profit{totalProfit}')

    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)

  if episode % 10 ==0:
    trader.model.save(f'mode_{episode}.h5')

  0%|          | 0/1257 [00:00<?, ?it/s]

Episode: 1/1000
AI Trader bought: $24.282499
AI Trader bought: $24.165001
AI Trader bought: $24.075001
AI Trader bought: $25.355000
AI Trader bought: $24.860001
AI Trader sold: 24.997499465942383 Profit: $0.715000
AI Trader sold: 23.522499084472656 Profit: - $0.642502
AI Trader bought: $24.334999
AI Trader sold: 24.107500076293945 Profit: $0.032499
AI Trader sold: 24.149999618530273 Profit: - $1.205000
AI Trader bought: $23.504999
AI Trader sold: 23.5674991607666 Profit: - $1.292501
AI Trader bought: $23.424999
AI Trader sold: 24.15999984741211 Profit: - $0.174999
AI Trader bought: $24.065001
AI Trader bought: $24.010000
AI Trader sold: 24.219999313354492 Profit: $0.715000
AI Trader sold: 23.672500610351562 Profit: $0.247501
AI Trader bought: $24.025000
AI Trader bought: $24.190001
AI Trader bought: $24.227501
AI Trader sold: 25.375 Profit: $1.309999


  3%|▎         | 35/1257 [00:11<33:08,  1.63s/it]

AI Trader sold: 25.25749969482422 Profit: $1.247499


  3%|▎         | 36/1257 [00:14<41:44,  2.05s/it]

...

AI Trader bought: $24.475000


  7%|▋         | 90/1257 [02:55<56:51,  2.92s/it]

AI Trader sold: 24.905000686645508 Profit: $1.435001


  7%|▋         | 91/1257 [02:58<56:46,  2.92s/it]

AI Trader sold: 25.102500915527344 Profit: $0.995001
